In [ ]:
import os
ss = []
for i in os.walk("../"):
    for j in i[2]:
        if ".ipynb" in j and "-chechpoint" not in j:
            path = i[0] + "/" + j
            if "table_of_content" not in path:
                ss.append(path)

In [ ]:
for f in ss:
    ! jupyter nbconvert --to=html --template=github.tpl "$f"

In [103]:
from bs4 import BeautifulSoup
import os, time
LINK_HTML = '<li class="breadcrumb-item"><a href="{link}">{name}</a></li>'

ENTRY_HTML = """
<div class="col-lg-4 ol-md-6 mb-2">
          <a href="{link}" class="text-dark">
          <div class="boxed p-2 rising">
            <div class="row align-items-center justify-content-between">
              <div class="col-10">
                <div class="media align-items-center">
                  <h4><i class="icon-{icon} mr-2"></i></h4>
                  <div class="media-body">
                    <h6 class="mb-0">{name}</h6>
                    <span class="text-muted">{mtime}</span>
                  </div>
                </div>
              </div>
            </div>
          </div>
        </a>
        </div>
"""

EXCLUDE = ['ipynb', 'assets', 'index.html', 'tpl', '.git', 'README', 'table_of_content']

In [104]:
def format_entry(i):
    for exc in EXCLUDE:
        if exc in i:
            return None
        
    _, fext = os.path.splitext(i)
    icon = "code"
    type_ = "Regular File"

    mtime = os.path.getmtime(i)
    mtime = time.strftime("%X %x", time.localtime(mtime))
    link = "./" + os.path.basename(i)

    if os.path.isdir(i):
        icon = "folder"
        type_ = "Folder"
        link += "/index.html"
    elif fext.lower() == ".html":
        icon = "book"
        type_ = "Notebook"
    elif fext.lower() == ".pdf":
        icon = "file-text"
        type_ = "PDF File"

    return ENTRY_HTML.format(link=link, icon=icon, 
                             name=os.path.basename(i), mtime=mtime)

In [105]:
def create_index(dir_path, template):
    
    
    html = open(template).read()
    soup = BeautifulSoup(html)
    path = soup.find(id="path")
    dentry = soup.find(id="dentry")
    absp = os.path.abspath(dir_path).split("/")
    absindex = absp.index('UofT-Course-Material-Repo')
    
    if absindex != -1 and len(absp[absindex:]) >= 2:
        end = absindex + 2
        while end <= len(absp):
            link = "/" + "/".join(absp[absindex: end]) + "/index.html"
            name = absp[end-1]
            insert_string = LINK_HTML.format(link=link, name=name)
            insert_soup = BeautifulSoup(insert_string, 'html.parser')
            path.append(insert_soup)
            end += 1

    for i in os.listdir(dir_path):
        insert_string = format_entry(dir_path + i)
        if insert_string:
            insert_soup = BeautifulSoup(insert_string, 'html.parser')
            dentry.append(insert_soup)
            
    return soup

In [106]:
folder_set = set([i[0] for i in os.walk("../")])
all_folder = []
for i in folder_set:
    if "../." not in i and "table_of_content" not in i:
        all_folder.append(i+ "/")
        
        
for i in all_folder:
    index = create_index(i, "./template.html")
    with open(i + "/index.html", "w") as f:
        f.write(str(index))